<a href="https://colab.research.google.com/github/minu-97/TIL/blob/master/bart_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.5 MB/s eta 0:00:00


In [4]:

!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np
import os
import json

data_list = []
topic_list = []
data_path = '/content/drive/MyDrive/multi/final_project/sodirty/data/'
forders = os.listdir(data_path)
print(len(forders))
for i in range(0,len(forders)):
    if forders[i].split('.')[1] == 'json':
        file = data_path+forders[i]
        with open(file, 'r') as file:
            data = json.load(file)
            data_str = ''.join(data['sentences'])
            data_list.append(data_str)
            topic = []
            for i in data['summaries']:
                topic.append(data['sentences'][i])
            topic_str = ''.join(topic)
            topic_list.append(topic_str)
            
# dict 형태로 합치기
test_dataset = data_list
target_text = topic_list

50


In [6]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    # GPU를 디바이스로 설정
    device = torch.device("cuda")
else:
    # CPU를 디바이스로 설정
    device = torch.device("cpu")

In [7]:
%%time
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

# tkenizer와 model불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
model_out = []

for i in range(0,len(test_dataset)):
  text = test_dataset[i]
  text = text.replace('\n', ' ')
  model.to(device)
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  max_length = 512  

  if len(input_ids) > max_length:
      input_ids = input_ids[:max_length]

  summary_ids = model.generate(torch.tensor([input_ids]).to(device),  num_beams=4,  max_length=512,  eos_token_id=1)
  summary_news = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

  model_out.append(summary_news)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


CPU times: user 1min 8s, sys: 3.24 s, total: 1min 11s
Wall time: 1min 20s


In [8]:
model_out

['미국 국방부는 지난달 말 인공지능(AI)을 이용해 인간 도움 없이 적을 식별해 타격하는 드론(무인 항공기)을 시연했다이 드론은 카메라 화면에서 총으로 무장한 사람과 무기가 없는 사람을 구분할 수 있다표적으로 정한 사람을 찾아 그가 탄 자동차를 추적하는 기능도 있다이 드론은 원격 조종 없이도 전장에서 특수부대 군인들처럼 임무를 수행하는 드론이 등장할 전망이다이 드론이 사람 도움 없이 카메라 영상에서 목표물을 인식하고 추적할 수 있는 것은 바로 ‘머신러닝’ 덕분이다머신러닝은 AI의 한 분야로 컴퓨터가 인간처럼 스스로 학습할 수 있는 능력을 부여하는 작업을 말한다머신러닝의 원리는 인간을 포함한 영장류 두뇌의 정보 처리 구조인 ‘신경망’을 모사하는 방식이다이세돌 9단을 이긴 구글의 ‘알파고’ 등 지금까지 소개된 AI 대부분은 심층신경망을 기반으로 한 머신러닝 알고리즘을 이용한다이미지에서 특정 사물을 찾는 기술은 인간이 아니라 고양이 뇌에서 유래했다고양이 뇌의 시신경에서 발견되는 ‘나선형 신경망’ 구조는 3단계에 걸쳐 점차적으로 단순화하면서 물체의 색깔이나 모양을 파악한다이를 처음으로 연구한 데이비드 휴벨과 토어스텐 비젤은 1981년 노벨 생리의학상을 받았다이를 처음으로 연구한 데이비드 휴벨과 토어스텐 비젤은 1981년 노벨 생리의학상을 받았다AI 과학자들은 나선형 신경망에서 아이디어를 얻어 사물을 판별하는 알고리즘을 설계했다우선 이미지에서 큰 특징을 추출한 다음 점차 작고 복잡한 특징을 발견해 나가는 방식이다이미지에서 사물을 찾는 기술은 인간이 아니라 고양이 뇌에서 유래했다고양이 뇌의 시신경에서 보내오는 반응을 모아 여러 개의 층(層)으로 나눈다이를 3단계에 걸쳐 점차적으로 단순화하면서 물체의 색깔이나 모양을 파악한다이를 처음으로 연구한 데이비드 휴벨과 토어스텐 비젤은 1981년 노벨 생리의학상을 받았다AI 과학자들은 나선형 신경망에서 아이디어를 얻어 이미지에서 사물을 판별하는 알고리즘을 설계했다우선 이미지에서 큰 특징을 추출한 다음 점차 작고 복잡한 특징을 발견

In [11]:
from rouge import Rouge
# avg 구하기 전 변수 초기화
total_1_r = 0
total_1_p = 0
total_1_f = 0
total_2_r = 0
total_2_p = 0
total_2_f = 0
total_l_r = 0
total_l_p= 0
total_l_f = 0
for i in range(0, len(model_out)):
  reference = [target_text[i]]
  rouge = Rouge()
  t= rouge.get_scores([model_out[i]], reference , avg = True)
  total_1_r += t['rouge-1']['r']
  total_1_p += t['rouge-1']['p']
  total_1_f += t['rouge-1']['f']
  total_2_r += t['rouge-2']['r']
  total_2_p += t['rouge-2']['p']
  total_2_f += t['rouge-2']['f']
  total_l_r += t['rouge-l']['r']
  total_l_p += t['rouge-l']['p']
  total_l_f += t['rouge-l']['f']
total_dict ={'rouge-1':{'r':total_1_r/50, 'p':total_1_p/50,'f':total_1_f/50},
             'rouge-2':{'r':total_2_r/50, 'p':total_2_p/50,'f':total_2_f/50},
             'rouge-l':{'r':total_l_r/50, 'p':total_l_p/50,'f':total_l_f/50}}

In [12]:
total_dict

{'rouge-1': {'r': 0.347137309101214,
  'p': 0.637175449670549,
  'f': 0.4010670140277458},
 'rouge-2': {'r': 0.2972523408334586,
  'p': 0.5708361371721667,
  'f': 0.34300781955581067},
 'rouge-l': {'r': 0.33095167935061626,
  'p': 0.5986306158902446,
  'f': 0.37976629744619006}}